# Spark SQL Lab

## 1. Thiết lập SQLContext

**Chú ý**: Để thực thi được các đoạn mã kết nối đến CSDL Teach (SQLite) ở bên dưới bạn cần chuyển đến thư mục làm việc từ miniconda prompt, activate conda enviromment có cài spark và thực thi lệnh sau để mở jupyter notebook/lab:

`pyspark --conf spark.executor.extraClassPath=sqlite-jdbc-3.43.0.0.jar --driver-class-path sqlite-jdbc-3.43.0.0.jar --jars sqlite-jdbc-3.43.0.0.jar`

In [9]:
pip install findspark

  Using cached findspark-2.0.1-py2.py3-none-any.whl.metadata (352 bytes)
Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: C:\Users\HUYNH THIEN NHAN\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip


In [12]:
import findspark
findspark.init()

import pyspark
findspark.find()

'C:\\spark'

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
           .config('spark.jars.packages', 'org.xerial:sqlite-jdbc:3.34.0')\
           .getOrCreate()

## 2. Tạo các dataframe từ các table

Ở bài lab này ta sẽ làm việc với CSDL `Teach`. Ta sẽ kết nối đến CSDL `Teach` (SQLite) và tạo ra các dataframe từ các table của CSDL này.

Lược đồ CSDL Teach gồm các bảng sau:

- `GiangVien(MaGV, TenGV, ChucDanh, Luong)`
- `MonHoc(MaMH, TenMH, SoTC)`
- `GiangDay(MaMH, HocKy, MaGV)`

**Ghi chú**: Trường `ChucDanh` có thể nhận các giá trị là `'Full Professor'`, `'Associate Professor'`, và `'Assistant Professor'`.

In [16]:
gv = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='GiangVien',
                 url='jdbc:sqlite:D:/HCMUTE/Namtu/BDML/lab2/teach.db')\
        .load()

In [17]:
gv.printSchema()

root
 |-- MaGV: string (nullable = true)
 |-- TenGV: string (nullable = true)
 |-- ChucDanh: string (nullable = true)
 |-- Luong: decimal(10,2) (nullable = true)



In [18]:
mh = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='MonHoc',
                 url='jdbc:sqlite:D:/HCMUTE/Namtu/BDML/lab2/teach.db')\
        .load()

In [19]:
mh.printSchema()

root
 |-- MaMH: string (nullable = true)
 |-- TenMH: string (nullable = true)
 |-- SoTC: long (nullable = true)



In [20]:
gd = spark.read.format('jdbc') \
        .options(driver='org.sqlite.JDBC', dbtable='GiangDay',
                 url='jdbc:sqlite:D:/HCMUTE/Namtu/BDML/lab2/teach.db')\
        .load()

In [21]:
gd.printSchema()

root
 |-- MaMH: string (nullable = true)
 |-- HocKy: string (nullable = true)
 |-- MaGV: string (nullable = true)



## 3. Tạo các temp view cho các dataframe

In [22]:
gv.createOrReplaceTempView("GiangVien")
mh.createOrReplaceTempView("MonHoc")
gd.createOrReplaceTempView("GiangDay")

## 4. Xem dữ liệu của các table

### 4.1. Xem dữ liệu bảng `GiangVien`
#### 4.1.1. Dùng Spark operation

In [23]:
gv.show()

+------+--------------+-------------------+------+
|  MaGV|         TenGV|           ChucDanh| Luong|
+------+--------------+-------------------+------+
|PROF01|  Sheldon Ross|Assistant Professor| 75.00|
|PROF02|Jeffrey Ullman|Assistant Professor| 90.00|
|PROF03|   Jon Patrick|     Full Professor|135.00|
|PROF04|  Joseph Davis|Associate Professor|120.00|
|PROF05| Gibert Strang|Associate Professor|120.00|
|PROF06|  Peter Norton|Assistant Professor| 90.00|
|PROF07| Sanjay Chawla|     Full Professor|140.00|
+------+--------------+-------------------+------+



#### 4.1.2. Dùng Spark SQL

In [24]:
query = """ 
    select *
    from GiangVien
"""
spark.sql(query).show()

+------+--------------+-------------------+------+
|  MaGV|         TenGV|           ChucDanh| Luong|
+------+--------------+-------------------+------+
|PROF01|  Sheldon Ross|Assistant Professor| 75.00|
|PROF02|Jeffrey Ullman|Assistant Professor| 90.00|
|PROF03|   Jon Patrick|     Full Professor|135.00|
|PROF04|  Joseph Davis|Associate Professor|120.00|
|PROF05| Gibert Strang|Associate Professor|120.00|
|PROF06|  Peter Norton|Assistant Professor| 90.00|
|PROF07| Sanjay Chawla|     Full Professor|140.00|
+------+--------------+-------------------+------+



### 4.2. Xem dữ liệu bảng `MonHoc`
#### 4.2.1. Dùng Spark operation

In [29]:
mh.show()

+-----+--------------------+----+
| MaMH|               TenMH|SoTC|
+-----+--------------------+----+
|CRS01|      Linear Algebra|   5|
|CRS02|Probability and S...|   5|
|CRS03|Discrete Mathamatics|   5|
|CRS04|Introduction to C...|   5|
|CRS05|            Database|   4|
|CRS06|Data Struture and...|   4|
|CRS07|Object Oriented P...|   4|
|CRS08|   Computer Networks|   4|
|CRS09|   Operating Systems|   4|
+-----+--------------------+----+



#### 4.2.2. Dùng Spark SQL

In [30]:
query = """ 
    select *
    from MonHoc
"""
spark.sql(query).show()

+-----+--------------------+----+
| MaMH|               TenMH|SoTC|
+-----+--------------------+----+
|CRS01|      Linear Algebra|   5|
|CRS02|Probability and S...|   5|
|CRS03|Discrete Mathamatics|   5|
|CRS04|Introduction to C...|   5|
|CRS05|            Database|   4|
|CRS06|Data Struture and...|   4|
|CRS07|Object Oriented P...|   4|
|CRS08|   Computer Networks|   4|
|CRS09|   Operating Systems|   4|
+-----+--------------------+----+



### 4.3. Xem dữ liệu bảng `GiangDay`
#### 4.3.1. Dùng Spark operation

In [27]:
gd.show()

+-----+-----+------+
| MaMH|HocKy|  MaGV|
+-----+-----+------+
|CRS02|F2006|PROF01|
|CRS02|S2006|PROF01|
|CRS03|S2004|PROF06|
|CRS04|F2006|PROF06|
|CRS04|F2009|PROF06|
|CRS04|F2010|PROF06|
|CRS05|F2005|PROF02|
|CRS06|F2008|PROF07|
|CRS07|F2010|PROF03|
|CRS08|F2010|PROF03|
|CRS08|S2006|PROF04|
|CRS09|F2006|PROF06|
|CRS09|S2006|PROF07|
+-----+-----+------+



#### 4.3.2. Dùng Spark SQL

In [28]:
query = """ 
    select *
    from GiangDay
"""
spark.sql(query).show()

+-----+-----+------+
| MaMH|HocKy|  MaGV|
+-----+-----+------+
|CRS02|F2006|PROF01|
|CRS02|S2006|PROF01|
|CRS03|S2004|PROF06|
|CRS04|F2006|PROF06|
|CRS04|F2009|PROF06|
|CRS04|F2010|PROF06|
|CRS05|F2005|PROF02|
|CRS06|F2008|PROF07|
|CRS07|F2010|PROF03|
|CRS08|F2010|PROF03|
|CRS08|S2006|PROF04|
|CRS09|F2006|PROF06|
|CRS09|S2006|PROF07|
+-----+-----+------+



## 5. Câu hỏi

Viết truy vấn SQL để trả lời các câu hỏi sau đồng thời thực thi chúng để hiển thị kết quả.

### Q1. Liệt kê tên giảng viên chưa từng dạy môn `'CRS07'`.

In [37]:
query = """ 
    select distinct TenGV
    from GiangDay as gd
    join GiangVien as gv
    on gv.MaGV = gd.MaGV
    where gd.MaMH <> 'CRS07'
"""
spark.sql(query).show()

+--------------+
|         TenGV|
+--------------+
|  Joseph Davis|
| Sanjay Chawla|
|Jeffrey Ullman|
|  Sheldon Ross|
|   Jon Patrick|
|  Peter Norton|
+--------------+



### Q2. Liệt kê tên giảng viên đã dạy cả hai môn `'CRS07'` và `'CRS08'`.

In [55]:
query = """ 
    select gv.TenGV
    from GiangVien as gv
    join GiangDay as gd1
    on gv.MaGV = gd1.MaGV
    join GiangDay as gd2
    on gv.MaGV = gd2.MaGV
    where gd1.MaMH = 'CRS07' and gd2.MaMH = 'CRS08'
    group by gv.TenGV
"""
spark.sql(query).show()

+-----------+
|      TenGV|
+-----------+
|Jon Patrick|
+-----------+



### Q3. Liệt kê tên giảng viên đã dạy cả hai môn `'CRS07'` và `'CRS08'` trong một học kỳ.

In [58]:
query = """ 
    select distinct gv.TenGV
    from GiangVien as gv
    join GiangDay as gd1
    on gv.MaGV = gd1.MaGV
    join GiangDay as gd2
    on gv.MaGV = gd2.MaGV
    where gd1.MaMH = 'CRS07'
      and gd2.MaMH = 'CRS08'
      and gd1.HocKy = gd2.HocKy
"""
spark.sql(query).show()

+-----------+
|      TenGV|
+-----------+
|Jon Patrick|
+-----------+



### Q4. Liệt kê tên giảng viên đã dạy môn `'CRS08'` nhưng chưa dạy môn `'CRS07'`.

In [59]:
query = """ 
    select distinct gv.TenGV
    from GiangVien as gv
    join GiangDay as gd1
    on gv.MaGV = gd1.MaGV
    where gd1.MaMH = 'CRS08'
    and gv.MaGV not in (
        select gd2.MaGV
        from GiangDay as gd2
        where gd2.MaMH = 'CRS07'
    )
"""
spark.sql(query).show()

+------------+
|       TenGV|
+------------+
|Joseph Davis|
+------------+



### Q5. Liệt kê tên môn học chỉ được dạy bởi những giảng viên là `'Full Professor'`.

In [64]:
query = """ 
    select distinct gd.MaMH
    from GiangDay as gd
    join GiangVien as gv
    on gd.MaGV = gv.MaGV
    where gv.ChucDanh = 'Full Professor'
      and not exists (
        select 1
        from GiangDay as gd2
        join GiangVien as gv2
        on gd2.MaGV = gv2.MaGV
        where gd.MaMH = gd2.MaMH
          and gv2.ChucDanh <> 'Full Professor'
      )
"""
spark.sql(query).show()

+-----+
| MaMH|
+-----+
|CRS06|
|CRS07|
+-----+



### Q6. Liệt kê tên môn được dạy bởi ít nhất hai giảng viên khác nhau.

In [65]:
query = """
    select gd.MaMH
    from GiangDay as gd
    group by gd.MaMH
    having count(distinct gd.MaGV) >= 2
"""
spark.sql(query).show()

+-----+
| MaMH|
+-----+
|CRS08|
|CRS09|
+-----+



### Q7. Liệt kê tên giảng viên có lương cao thứ hai.

In [66]:
query = """
    with RankedSalaries as (
        select gv.TenGV, gv.Luong,
               row_number() over (order by gv.Luong desc) as rank
        from GiangVien as gv
    )
    select TenGV
    from RankedSalaries
    where rank = 2
"""
spark.sql(query).show()

+-----------+
|      TenGV|
+-----------+
|Jon Patrick|
+-----------+



### Q8. Liệt kê tên tên giảng viên dạy nhiều môn nhất trong học kỳ `'F2010'`.

In [67]:
query = """
    with TeacherSubjectCount as (
        select gv.TenGV, count(distinct gd.MaMH) as SubjectCount
        from GiangDay as gd
        join GiangVien as gv
        on gd.MaGV = gv.MaGV
        where gd.HocKy = 'F2010'
        group by gv.TenGV
    )
    select TenGV
    from TeacherSubjectCount
    where SubjectCount = (
        select max(SubjectCount)
        from TeacherSubjectCount
    )
"""
spark.sql(query).show()

+-----------+
|      TenGV|
+-----------+
|Jon Patrick|
+-----------+



### Q9. Liệt kê tên (những) giảng viên dạy tất cả các môn được mở trong học kỳ `'S2004'`.

In [68]:
query = """
    select gv.TenGV
    from GiangDay as gd
    join GiangVien as gv
    on gd.MaGV = gv.MaGV
    where gd.HocKy = 'S2004'
    group by gv.TenGV
    having count(distinct gd.MaMH) = (
        select count(distinct gd2.MaMH)
        from GiangDay as gd2
        where gd2.HocKy = 'S2004'
    )
"""
spark.sql(query).show()

+------------+
|       TenGV|
+------------+
|Peter Norton|
+------------+



### Q10. Liệt kê tên (những) giảng viên đã dạy nhiều môn hơn số môn trung bình được dạy bởi mỗi giảng viên trong tất cả các học kỳ.

In [69]:
query = """
    select gv.TenGV
    from GiangDay as gd
    join GiangVien as gv
    on gd.MaGV = gv.MaGV
    group by gv.TenGV
    having count(distinct gd.MaMH) > (
        select avg(CourseCount)
        from (
            select count(distinct gd2.MaMH) as CourseCount
            from GiangDay as gd2
            group by gd2.MaGV
        ) as AvgCourses
    )
"""
spark.sql(query).show()

+-------------+
|        TenGV|
+-------------+
|Sanjay Chawla|
|  Jon Patrick|
| Peter Norton|
+-------------+

